In [1]:
import pandas as pd

# Load the dataset
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')


In [2]:
df_cap.head()

,Facility,Max_Capacity
0,0,15060
1,1,15491
2,2,18463
3,3,15757
4,4,18780


In [3]:
df_de.head()

,Region,Demand
0,0,5157
1,1,2449
2,2,6580
3,3,5375
4,4,2727


In [4]:
df_cost.head()

,Facility,Fixed_Cost,Revenue_Region_1,Revenue_Region_2,Revenue_Region_3,Revenue_Region_4,Revenue_Region_5,Revenue_Region_6,Revenue_Region_7,Revenue_Region_8,...,Revenue_Region_22,Revenue_Region_23,Revenue_Region_24,Revenue_Region_25,Revenue_Region_26,Revenue_Region_27,Revenue_Region_28,Revenue_Region_29,Revenue_Region_30,Revenue_Region_31
0,0,30977216,168,165,370,289,139,275,212,492,...,320,473,422,353,381,76,450,64,70,462
1,1,19988490,214,302,221,314,374,195,210,187,...,81,305,94,469,270,299,84,442,71,390
2,2,46375256,235,167,275,294,377,303,107,422,...,119,402,469,118,199,364,101,448,458,456
3,3,33623823,291,390,464,487,419,336,56,284,...,398,398,79,338,443,57,94,490,265,346
4,4,45544254,186,150,209,307,263,371,348,207,...,429,101,272,398,237,222,214,52,326,428


## Question 1

### (a) What condition(s) must be met for the optimal objective function value to be positive?

#### For the optimal objective function value to be positive, the total revenue from distributing brakes must exceed the total fixed cost of establishing the distribution centers. This requires that at least one DC is opened and that the volume of brakes shipped from the open facility or facilities—when multiplied by the corresponding per-unit revenues—is high enough to cover the fixed cost of operating those centers. Essentially, there must exist a feasible solution where the revenue generated from the shipments outweighs the fixed investment required to set up the network, ensuring a net profit greater than zero.

### (b) Interpret the constraint that links the decision to open a distribution center with their distribution plan, given that the model addresses a capacity planning problem over the next 10 years.

#### The linking constraint, formulated as x[i, j] ≤ 0.45 * Demand[j] * y[i], serves a dual purpose. First, it ensures that if a distribution center is not opened (i.e., y[i] = 0), then no brakes can be distributed from that facility (forcing x[i, j] to zero for all regions j). Second, it limits the volume that can be shipped from an open facility to any region to at most 45% of that region’s demand. This linkage is critical in a 10-year capacity planning problem because it ties the irreversible investment decision of opening a facility to its operational use. Essentially, it guarantees that only facilities that are built can contribute to the distribution plan, and even then, they must operate within realistic bounds that reflect regional demand constraints.

### (c) Benchmarking is useful for providing context. Therefore using Gurobi and assuming constraints 1−8 above are omitted, what would the optimal profit be?

In [2]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data files
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists for facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, and fixed costs
# Assuming capacity is the annual capacity
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build a revenue dictionary:
# Revenue columns in df_cost are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'.
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# Create a new Gurobi model for the benchmark problem (omitting constraints 1-8)
model = Model("Benchmark_10Year")
model.setParam('OutputFlag', 1)  # Enable solver output

# Decision Variables:
# y[i] is a binary variable indicating whether facility i is opened.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] is a continuous variable representing the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# Basic Constraints:

# 1. Capacity Constraint: The total annual shipments from facility i cannot exceed its annual capacity,
#    and shipments are only allowed if the facility is open.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# 2. Demand Constraint: The total annual shipments to region j cannot exceed its annual demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# Objective Function:
# Calculate the 10-year total revenue (annual revenue multiplied by 10) minus the fixed cost (incurred once).
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Print the optimal objective value (10-year profit)
print("Optimal 10-Year Profit (Benchmark):", model.objVal)


Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 48 rows, 544 columns and 1071 nonzeros
Model fingerprint: 0x614e2a46
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 48 rows, 544 columns, 1071 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)

Root relaxation: objective 4.974484e+08, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.9745e+08    0    6   -0.00000 4.9745e+08      -    

#### The optimal profit is 483175185.0

### (d) After solving the full MILP model with Gurobi, what is the optimal profit?

In [3]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load the data files
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists for facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16 (0-indexed)
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, fixed costs
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build revenue dictionary:
# Revenue columns in df_cost are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# Create a new Gurobi model for the full MILP model (10-year profit)
model = Model("FullMILP_10Year")
model.setParam('OutputFlag', 1)  # Enable solver output

# Decision Variables:
# y[i] is a binary variable indicating whether facility i is opened.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] is a continuous variable representing the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# ----------------------------
# Basic Constraints
# ----------------------------

# (1) Capacity Constraint: 
# For each facility i, the total annual shipments cannot exceed its capacity if the facility is open.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# (2) Demand Constraint: 
# For each region j, the total annual shipments received cannot exceed its annual demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# ----------------------------
# Strategic & Operational Constraints (Full MILP)
# ----------------------------

# Note: The problem statement provides locations in 1-indexed form.
# We convert them to 0-indexed based on the CSV files (Facility 0 corresponds to location 1, etc.).

# Constraint 1: At most 3 DCs can be opened at locations {1,2,3,7,11,13,17}.
# 1-indexed locations -> 0-indexed: {0, 1, 2, 6, 10, 12, 16}.
A = [0, 1, 2, 6, 10, 12, 16]
model.addConstr(quicksum(y[i] for i in A) <= 3, name="Constraint1")

# Constraint 2: If a DC is established at location 4 then facilities must also be opened at locations 6,8,10.
# 1-indexed: location 4 -> index 3; locations 6,8,10 -> indices 5,7,9.
model.addConstr(y[3] <= y[5], name="Constraint2a")
model.addConstr(y[3] <= y[7], name="Constraint2b")
model.addConstr(y[3] <= y[9], name="Constraint2c")

# Constraint 3: If a DC is opened at location 2 then at least two DCs must be opened at locations {12,14,16}.
# 1-indexed: location 2 -> index 1; locations 12,14,16 -> indices 11,13,15.
model.addConstr(y[11] + y[13] + y[15] >= 2 * y[1], name="Constraint3")

# Constraint 4: At most 2 DCs can be opened at locations {1,8,9,17}.
# 1-indexed: {1,8,9,17} -> 0-indexed: {0,7,8,16}.
model.addConstr(quicksum(y[i] for i in [0, 7, 8, 16]) <= 2, name="Constraint4")

# Constraint 5: If a DC is opened at location 1 then a facility must be established at location 5 or 6.
# 1-indexed: location 1 -> index 0; locations 5 and 6 -> indices 4 and 5.
model.addConstr(y[0] <= y[4] + y[5], name="Constraint5")

# Constraint 6: The number of DCs opened at locations 1 through 9 must not exceed 1.2 times the number of DCs
# opened at locations 10 through 17.
# 1-indexed: locations 1-9 -> indices 0 to 8; locations 10-17 -> indices 9 to 16.
model.addConstr(quicksum(y[i] for i in range(0, 9)) <= 1.2 * quicksum(y[i] for i in range(9, 17)), name="Constraint6")

# Constraint 7: The total annual shipments from the first 6 DCs (locations 1-6) must be at least 39% of the total annual shipments.
# 1-indexed: locations 1-6 -> indices 0 to 5.
model.addConstr(
    quicksum(x[i, j] for i in range(0, 6) for j in regions) >= 0.39 * quicksum(x[i, j] for i in facilities for j in regions),
    name="Constraint7"
)

# Constraint 8: Linking Constraint – shipments from facility i to region j are allowed only if facility i is open,
# and each shipment cannot exceed 45% of the region's annual demand.
model.addConstrs(
    (x[i, j] <= 0.45 * demand[j] * y[i] for i in facilities for j in regions),
    name="Constraint8"
)

# ----------------------------
# Objective Function
# ----------------------------
# The model maximizes the 10-year profit.
# Annual revenue is multiplied by 10, while fixed costs are incurred only once.
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Print the optimal objective value (10-year profit)
print("Optimal 10-Year Profit (Full MILP):", model.objVal)


Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 584 rows, 544 columns and 2693 nonzeros
Model fingerprint: 0xc82b3982
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 584 rows, 544 columns, 2693 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)

Root relaxation: objective 4.392851e+08, 183 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.3929e+08    0    7   -0.00000 4.3929e+08      - 

### (e) Discuss the value of benchmarking by comparing the optimal objective function values from the previous two questions. What managerial insights can be drawn from this comparison?

#### Benchmarking offers a valuable reference point by showing the maximum potential profit without practical constraints. In our model, when all strategic and operational restrictions are omitted, the optimal 10-year profit is approximately \$483 million. However, once all constraints are included, the profit decreases to about \$424 million. This roughly \$59 million gap highlights the trade-offs that come with implementing real-world restrictions—such as location limits, dependency requirements, and shipment caps. For managers, this comparison emphasizes that while a theoretical model might suggest very high profitability, the necessary operational constraints significantly lower this figure. Understanding the cost imposed by these constraints can guide decision-makers in evaluating whether some restrictions might be adjusted or if investments should be made to mitigate their impact, ultimately helping to balance profitability with practical considerations.

### (f) If we solved the linear relaxation of this problem, it would be classified as a linear program. Although this would simplify the problem, the resulting optimal solution would not be integral. Explain why using this method to solve the problem faster is not practical in this case.

#### Solving the linear relaxation would indeed simplify the computation by allowing fractional values for the decision variables, but this approach is not practical in our case because the model's key decisions are inherently discrete. In our MILP, variables like \(y_i\) represent the decision to open a facility, and they must be binary—either a facility is opened (1) or not (0). When relaxed, these values could fall between 0 and 1 (for example, 0.6), which does not have any real-world interpretation (a facility cannot be "partially" open). Moreover, rounding these fractional solutions to obtain an integer solution could lead to infeasibilities or suboptimal outcomes due to the tight coupling between fixed costs, capacities, and shipping decisions. In essence, although LP relaxation provides a computationally faster bound on the objective function, the resulting solution would not be directly implementable for strategic capacity planning over the 10-year period, making the use of full MILP methods necessary to capture the discrete, combinatorial nature of the problem.

### (g) For the full MILP model, how many feasible solutions are within 1% of the optimal solution? What is the smallest objective function value of the feasible solution(s) you have found?

In [4]:
# Set the Gurobi parameters to populate the solution pool.
# PoolSearchMode=2 instructs Gurobi to search for multiple solutions,
# and PoolSolutions sets the maximum number of solutions to store.
model.setParam('PoolSearchMode', 2)
model.setParam('PoolSolutions', 1000)

# Re-optimize the model to populate the solution pool.
model.optimize()

# Retrieve the optimal objective value from the best (incumbent) solution.
opt_obj = model.objVal
# Define tolerance: a solution is within 1% if its objective value is at least 0.99 * opt_obj.
tol = 0.99 * opt_obj

# Initialize counters
num_within_1pct = 0
min_obj_within = float('inf')

# Loop through all solutions in the pool.
for i in range(model.SolCount):
    model.setParam('SolutionNumber', i)
    sol_obj = model.PoolObjVal
    # Check if the solution is within 1% of the optimal.
    if sol_obj >= tol:
        num_within_1pct += 1
        if sol_obj < min_obj_within:
            min_obj_within = sol_obj

print("Number of feasible solutions within 1% of optimal:", num_within_1pct)
print("Smallest objective value among these solutions:", min_obj_within)


Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1000
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  1000
PoolSearchMode  2

Optimize a model with 584 rows, 544 columns and 2693 nonzeros
Model fingerprint: 0xc82b3982
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+03]
Presolved: 584 rows, 544 columns, 2693 nonzeros

Continuing optimization...


Cutting planes:
  Lift-and-project: 1
  Cover: 1
  Implied bound: 6
  MIR: 20
  Flow cover: 23
  Zero half: 1
  Relax-and-lift: 3

Explored 1 nodes (405 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 4.23791e+08 4.1

In [5]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ----------------------------
# Data Loading and Setup
# ----------------------------
df_cap = pd.read_csv('/Users/Sam/Downloads/capacity.csv')
df_de  = pd.read_csv('/Users/Sam/Downloads/demand.csv')
df_cost = pd.read_csv('/Users/Sam/Downloads/costs_revenues.csv')

# Create lists of facilities and regions
facilities = df_cap['Facility'].tolist()   # Facilities: 0 to 16 (0-indexed)
regions = df_de['Region'].tolist()           # Regions: 0 to 30

# Build dictionaries for capacities, demands, and fixed costs
capacity = { int(row['Facility']): row['Max_Capacity'] for _, row in df_cap.iterrows() }
demand   = { int(row['Region']): row['Demand'] for _, row in df_de.iterrows() }
fixed_cost = { int(row['Facility']): row['Fixed_Cost'] for _, row in df_cost.iterrows() }

# Build a revenue dictionary
# Revenue columns are named 'Revenue_Region_1', 'Revenue_Region_2', ..., 'Revenue_Region_31'
revenue = {}
for idx, row in df_cost.iterrows():
    i = int(row['Facility'])
    for j in range(1, 32):
        revenue[(i, j-1)] = row[f'Revenue_Region_{j}']

# ----------------------------
# Create Model and Set Parameters for Solution Pool
# ----------------------------
model = Model("FullMILP_10Year_SolutionPool")
model.setParam('OutputFlag', 1)       # Enable solver output
model.setParam('PoolSearchMode', 2)     # Enable solution pool search
model.setParam('PoolSolutions', 100)    # Request up to 100 solutions

# ----------------------------
# Decision Variables
# ----------------------------
# y[i] = 1 if facility i is open, 0 otherwise.
y = model.addVars(facilities, vtype=GRB.BINARY, name="y")
# x[i,j] represents the annual shipments from facility i to region j.
x = model.addVars(facilities, regions, vtype=GRB.CONTINUOUS, lb=0, name="x")

# ----------------------------
# Basic Constraints
# ----------------------------
# (1) Capacity Constraint:
# For each facility i, the total annual shipments must not exceed its capacity if the facility is open.
model.addConstrs(
    (quicksum(x[i, j] for j in regions) <= capacity[i] * y[i] for i in facilities),
    name="Capacity"
)

# (2) Demand Constraint:
# For each region j, the total annual shipments received must not exceed the region's annual demand.
model.addConstrs(
    (quicksum(x[i, j] for i in facilities) <= demand[j] for j in regions),
    name="Demand"
)

# ----------------------------
# Strategic & Operational Constraints (Converted to 0-indexed)
# ----------------------------
# Constraint 1: At most 3 DCs can be opened at locations {1,2,3,7,11,13,17} (1-indexed -> indices {0,1,2,6,10,12,16})
A = [0, 1, 2, 6, 10, 12, 16]
model.addConstr(quicksum(y[i] for i in A) <= 3, name="Constraint1")

# Constraint 2: If a DC is established at location 4 (index 3), then facilities must also be opened at locations 6,8,10 (indices 5,7,9)
model.addConstr(y[3] <= y[5], name="Constraint2a")
model.addConstr(y[3] <= y[7], name="Constraint2b")
model.addConstr(y[3] <= y[9], name="Constraint2c")

# Constraint 3: If a DC is opened at location 2 (index 1), then at least two DCs must be opened at locations {12,14,16} (indices 11,13,15)
model.addConstr(y[11] + y[13] + y[15] >= 2 * y[1], name="Constraint3")

# Constraint 4: At most 2 DCs can be opened at locations {1,8,9,17} (1-indexed -> indices {0,7,8,16})
model.addConstr(quicksum(y[i] for i in [0, 7, 8, 16]) <= 2, name="Constraint4")

# Constraint 5: If a DC is opened at location 1 (index 0), then a facility must be established at location 5 or 6 (indices 4,5)
model.addConstr(y[0] <= y[4] + y[5], name="Constraint5")

# Constraint 6: The number of DCs opened at locations 1 through 9 (indices 0-8) must not exceed 1.2 times the number opened at locations 10 through 17 (indices 9-16)
model.addConstr(quicksum(y[i] for i in range(0, 9)) <= 1.2 * quicksum(y[i] for i in range(9, 17)), name="Constraint6")

# Constraint 7: The total annual shipments from the first 6 DCs (locations 1-6, indices 0-5) must be at least 39% of the total annual shipments.
model.addConstr(
    quicksum(x[i, j] for i in range(0, 6) for j in regions) >= 0.39 * quicksum(x[i, j] for i in facilities for j in regions),
    name="Constraint7"
)

# Constraint 8: Linking Constraint – shipments from facility i to region j are only allowed if facility i is open,
# and each shipment cannot exceed 45% of the region's annual demand.
model.addConstrs(
    (x[i, j] <= 0.45 * demand[j] * y[i] for i in facilities for j in regions),
    name="Constraint8"
)

# ----------------------------
# Objective Function
# ----------------------------
# Maximize 10-year profit: 10 times the annual revenue minus the fixed cost (incurred only once)
model.setObjective(
    10 * quicksum(revenue[(i, j)] * x[i, j] for i in facilities for j in regions)
    - quicksum(fixed_cost[i] * y[i] for i in facilities),
    GRB.MAXIMIZE
)

# ----------------------------
# Optimize the Model
# ----------------------------
model.optimize()

# Print the optimal objective value (10-year profit)
best_obj = model.objVal
print("Optimal 10-Year Profit (Full MILP):", best_obj)

# ----------------------------
# Retrieve and Analyze the Solution Pool for Question (g)
# ----------------------------
sol_count = model.SolCount
print("Number of solutions in the pool:", sol_count)

# Count the number of solutions within 1% of the optimal objective and find the smallest objective among them.
within_gap_count = 0
min_obj_within_gap = float('inf')
for i in range(sol_count):
    # Set the solution number to i to query that solution's objective value.
    model.setParam(GRB.Param.SolutionNumber, i)
    sol_obj = model.objVal  # Get objective value for the current solution
    # For a maximization problem, a solution is within 1% of optimal if its objective value is at least 99% of the best objective.
    if sol_obj >= 0.99 * best_obj:
        within_gap_count += 1
        if sol_obj < min_obj_within_gap:
            min_obj_within_gap = sol_obj

print("Number of solutions within 1% of the optimal solution:", within_gap_count)
print("Smallest objective value among these solutions:", min_obj_within_gap)


Set parameter OutputFlag to value 1
Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 100
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  100
PoolSearchMode  2

Optimize a model with 584 rows, 544 columns and 2693 nonzeros
Model fingerprint: 0xc82b3982
Variable types: 527 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+04]
  Objective range  [5e+02, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 8e+03]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 584 rows, 544 columns, 2693 nonzeros
Variable types: 527 continuous, 17 integer (17 binary)
Found heuristic solution: objective 3.903807e+08

Root relaxation: objective 4.392851e+08, 183 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curr

### (h) Finding both the optimal solution and several sub-optimal solutions within a relative optimality gap can be challenging for more complex problems. How can you configure Gurobi to return the first feasible solution it finds within a specified percentage of the optimal solution? Using this parameter setting, what is the objective function value for a solution within 5% of optimality?

### (i) Write down two additional Gurobi parameters that can be used to find feasible but sub-optimal solutions for difficult optimization problems, and briefly explain why they are applicable.

### (j) Explain why, in cases like this, using Gurobi parameters to solve an approximate version of an optimization problem is preferable to solving an LP relaxation of the same model.